# Clase 1 – Fundamentos y diagnóstico de rendimiento en Spark

## Objetivos
- Entender cómo ejecuta Spark un job
- Diferenciar transformaciones y acciones
- Analizar DAGs, stages y tasks
- Identificar shuffles y problemas de rendimiento


## Bloque 1 – Transformaciones vs Acciones

Spark trabaja de forma **lazy**:  
las transformaciones no se ejecutan hasta que aparece una acción.


### Transformaciones
- Operaciones que definen un nuevo DataFrame/RDD.
- _Lazy evaluation_: no ejecutan nada hasta que llega una acción.
- Son operaciones perezosas (_lazy evaluation_): no se ejecutan inmediatamente.
- Devuelven un nuevo DataFrame/RDD con un plan de ejecución actualizado, pero sin lanzar un job.
- Se ejecutan solo cuando se dispara una acción.
- Pueden ser de dos tipos:
    1. Narrow transformations: los datos de una partición se usan solo en esa misma partición.
        - Ejemplos: `map()`, `filter()`, `select()`, `withColumn()`.
        - Más eficientes (no requieren shuffle).
    2. Wide transformations: requieren mover datos entre particiones → shuffle.
        - Ejemplos: `groupBy()`, `join()`, `distinct()`, `repartition()`.
        - Costosas en tiempo y recursos.


Copia este fragmento en el Notebook
```python
from pyspark.sql.functions import col

df = spark.range(1, 100).withColumn("x2", col("id") * 2)  # transformación
df_filtered = df.filter(col("id") > 50)                   # transformación

# Hasta aquí no se ejecuta nada
```

### Acciones:
- Disparan la ejecución real del plan.
- Devuelven un resultado al driver o escriben datos en almacenamiento.
- Ejemplos:
  - `collect()` → trae todos los datos al driver (⚠️ cuidado con Out Of Memory).
  - `show()` → muestra primeras filas en consola.
  - `count()` → cuenta filas.
  - `write.format("delta").save(...)` → guarda en disco.


Copia este fragmento en el notebook y analiza
```python
df_filtered.show()   # aquí Spark ejecuta el pipeline
```


### Ejemplo completo

Copia este fragmento en el notebook y analiza
```python
# Transformaciones (lazy)
df = spark.range(1, 1000000)
df2 = df.withColumn("x2", col("id") * 2)   # narrow
df3 = df2.filter(col("x2") % 5 == 0)       # narrow
df4 = df3.groupBy((col("x2") % 10)).count() # wide → shuffle

# Acción (trigger)
df4.show()
```

%md
Resultado:
- Hasta `.groupBy()` solo hay transformaciones → Spark construye el DAG.
- Al llamar `.show()`, Spark ejecuta el job, lo divide en stages y tasks.

### Resumen clave
- Hasta que no hay una acción, Spark no ejecuta
- Varias transformaciones se encadenan en un mismo job


## Bloque 2 – DAG, Stages y Tasks
Spark divide los pipelines en DAGs → stages → tasks.
  
Entender esto nos hace comprender cómo afecta al paralelismo y en definitiva al rendimiento.

1. **DAG (Directed Acyclic Graph)**
   - Qué es: Representación del flujo de transformaciones como un grafo dirigido sin ciclos.
    - Cada nodo = operación (map, filter, join, etc.).
    - Cada arista = dependencia de datos entre operaciones.
    - Spark construye el DAG de manera perezosa (lazy evaluation).
    - El DAG no se ejecuta hasta que se encuentra una acción (`.show()`, `.collect()`, `.count()`).

2. **Stages**
   - El DAG se divide en stages según los puntos de shuffle.
   - Narrow dependency: cada partición depende solo de una partición anterior → mismo stage. Ejemplo: `map`, `filter`.
   - Wide dependency: una partición depende de varias → nuevo stage. Ejemplo: `groupBy`, `join`, `distinct`.

3. **Tasks**
   - La unidad más pequeña de trabajo en Spark.
   - Cada task procesa una partición de datos en un executor.
   - Ejemplo: si tengo 200 particiones → Spark lanza 200 tasks (distribuidas en los executors).


### Ejemplo DAG Complejo
Copia las siguientes celdas en el notebook y analiza lo que ocurre en cada caso

```python
from pyspark.sql.functions import col, rand

# Ajustamos las particiones de shuffle para que se generen muchas tasks
spark.conf.set("spark.sql.shuffle.partitions", 200)

# ==============================
# DataFrame base (50 millones filas, 200 particiones iniciales)
# ==============================
df = spark.range(0, 50_000_000, numPartitions=200).withColumn("value", (col("id") * rand()))

# ==============================
# JOB 1: count con 2 stages
# ==============================
# Stage 1: lectura + filtro (narrow)
df_filtered = df.filter(col("value") > 1000)

# Stage 2: shuffle por groupBy + count
df_grouped = df_filtered.groupBy((col("id") % 10).alias("bucket")).count()

# Acción que dispara Job 1
print("Job 1 result:", df_grouped.count())
```

```python

# Cuando llamamos a show(), Spark vuelve a ejecutar el pipeline porque es una acción.

df_grouped.show()
```

```python

# Cuando llamamos a cache(), Spark vuelve a ejecutar el pipeline porque es una acción.
# al igual que antes el job tiene 2 stages pero el segundo más largo con la instrucción de cache al final

df_grouped.cache()
```
Aviso: Aquí al ejecutarlo va a dar un fallo porque la version gratis de databricks no permite cachear el df

```python

# Cuando llamamos a show(), Spark lee de la cache el df y eso ahorra tiempo

df_grouped.show()
```

```python

# ==============================
# JOB 2: join con 3 stages
# ==============================
df1 = spark.range(0, 10_000_000, numPartitions=100).withColumn("k", col("id") % 5000)
df2 = spark.range(0, 20_000_000, numPartitions=150).withColumn("k", col("id") % 5000)

# Stage 1: lectura df1
# Stage 2: lectura df2
# Stage 3: shuffle para join y acción final (show)
df_joined = df1.join(df2, on="k", how="inner")

# Acción que dispara Job 2
df_joined.show(5)
```

### Ejercicio DAGs
Ve al Notebook y realiza los ejercicios de los DAGs

#### Spark UI
La Spark UI es la interfaz web que te muestra qué está pasando dentro de tu aplicación Spark. En Databricks puedes acceder desde la pestaña Spark UI de cada job o notebook.
Los elementos principales son:
1. Jobs tab
   - Lista de trabajos disparados por acciones (count(), show(), write...).
   - Cada job corresponde a la ejecución de un DAG.
   - Desde aquí entras a Stages.
2. Stages tab
   - Muestra cómo se divide el job en stages (fases).
   - Cada stage tiene múltiples tasks (una por partición).
   - Métricas clave: tiempo, duración, skew, GC, I/O.
3. Tasks tab (dentro de un stage)
   - Detalle de cada partición ejecutada.
   - Puedes detectar data skew: si una task tarda mucho más que las demás.
   - Métricas de input size, shuffle read/write, memoria usada.
4. SQL tab
   - Muy útil si usas DataFrames/SQL.
   - Visualiza el plan lógico y físico en forma de árbol.
   - Identifica dónde ocurren shuffles y scans de tablas. 
5. Storage tab
   - Muestra qué DataFrames/RDDs están cacheados.
   - Permite ver el uso de memoria y disco para persistencia.

En Databricks también tienes el DAG Viewer, un grafo visual que muestra stages y dependencias → muy útil para enseñar.

![Interfaz general del Spark UI](https://docs.databricks.com/aws/en/assets/images/spark-ui-1effb898029c011d46288302118e96ce.png)

![Spark UI para los stages](https://www.databricks.com/wp-content/uploads/2016/10/07-debug-spark-ui.png)

![](https://www.databricks.com/wp-content/uploads/2015/06/Screen-Shot-2015-06-19-at-2.00.59-PM.png)

#### `explained()`
El método `.explain()` muestra cómo Spark planea ejecutar tu DataFrame.
Opciones principales:
- `.explain()` → plan físico resumido.
- `.explain("extended")` → plan lógico + optimizado + físico.
- `.explain("cost")` → incluye estimación de costes (filas, bytes).
- `.explain("formatted")` → salida legible en tabla.

### Demo práctica
Copia este fragmento de código en el notebook y el resultado:
```python
# Crear DataFrame grande
df = spark.range(0, 10000000)

# Solo definimos transformaciones (lazy)
df_filtered = df.filter(df.id % 2 == 0)
df_transformed = df_filtered.withColumn("id_squared", df.id * df.id)

# Ver plan lógico/físico
df_transformed.explain("formatted")
```

Se debería ver algo parecido a esto:
1. PhotonRange (1)
   - Spark crea un dataset de enteros (de 0 a 10,000,000).
   - Está dividido en 8 splits (= particiones iniciales).
   - Paralelización inicial.
2. PhotonFilter (2)
   - Filtra solo los números pares: (id % 2 = 0).
   - Es un narrow transformation (no hay shuffle).
3. PhotonProject (3)
   - Calcula una nueva columna id_squared.
   - Otra transformación ligera, todavía sin shuffle.
4. PhotonResultStage (4)
   - Prepara los resultados para pasarlos al driver.
5. ColumnarToRow (5)
   - Convierte los datos del formato columnar interno (usado por Photon) a filas normales, porque el driver no entiende el formato columnar.

#### Ejercicio - Uso `explained()`
Ve al Notebook y realiza el ejercicio de esa sección

## Bloque 3 – Shuffles, Data Skew y Particiones

### Shuffles
  - Redistribución de datos entre particiones.
  - Se producen en `groupBy`, `join`, `distinct`, `orderBy`.
  - Costosos porque implican:
    - Escritura a disco (shuffle files).
    - Transferencia por red.
    - Lectura posterior.
  - ⚠️ Los shuffles son la principal fuente de cuellos de botella en Spark.



### Data Skew (desequilibrio de datos) 
  - Cuando una clave concentra muchos más datos que el resto.
  - Consecuencia: tareas desbalanceadas → unas rápidas, otras muy lentas.
  - Efecto visible: long tail tasks.
  - Ejemplo: 80% de las filas en la misma clave.

  Imagina que tienes un df formado de esta forma:
```python
N = 10_000_000
df = spark.range(0, N).withColumn(
    "skewed_key",
    when(col("id") < int(N*0.99), lit(1))   # 95% de las filas van con la clave "1"
    .otherwise((col("id") % 1000) + 2)      # el 5% restante se reparte
)

df_grouped = df.groupBy("skewed_key").count()
```
Va a ser un dataframe muy desbalanceado donde una sola clave (key = 0) concentra la mayoría de las filas y provocará que unas tareas terminan rápido y otras tardan muchísimo (long tail). 

Cuando se analicen sus tareas se verá así:
![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*qI2TxDDpwZ4bzL3Vu_B3Hg.png)

### Particiones y paralelismo
  - Spark divide el trabajo en particiones → tasks.
  - Demasiadas particiones → overhead administrativo, demasiados archivos pequeños.
  - Muy pocas particiones → subutilización de CPU, tareas gigantes que bloquean.
  - Config clave:
    - `spark.sql.shuffle.partitions` (por defecto: 200).
    - `repartition()` y `coalesce()`.

Para intentar corregir este problema se puede jugar con alguna de estas opciones:
- `repartition(n)`
- `coalesce(n)`
- `spark.sql.shuffle.partitions`

Prueba a ejecutar este fragmento de código para ver cómo varias las particiones cambiando el valor. 200 es el por defecto
```python
from pyspark.sql.functions import col, when, rand
spark.conf.set("spark.sql.shuffle.partitions", "200")  # valor inicial por defecto
```

  El rendimiento depende de encontrar el equilibrio justo.